# Find the city of each train station
In order to find the city of each train station we need a representation of the area of each city : https://public.opendatasoft.com/explore/dataset/georef-france-commune/export/?disjunctive.reg_name&disjunctive.dep_name&disjunctive.arrdep_name&disjunctive.ze2020_name&disjunctive.bv2012_name&disjunctive.epci_name&disjunctive.ept_name&disjunctive.com_name&disjunctive.ze2010_name&disjunctive.com_is_mountain_area&refine.dep_name=Paris&location=9,42.90011,2.31812&basemap=jawg.light

Here you can find a representation with the geojson format which is loadable with pandas_geojson.

```
pip install pandas_geojson

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

stops = pd.read_csv('data_sncf/stops.txt')

def formatID(id):
    return ''.join([i for i in id if i.isdigit()])

stops['stop_id']=stops['stop_id'].apply(formatID)
stops=stops.drop_duplicates(subset=['stop_id'])
stops=stops.dropna(subset=['stop_name'])

region = gpd.read_file('georef-france-region/georef-france-region-millesime.shp',encoding='utf-8')
region.loc[(region['reg_is_ctu']=='Non') & (region['reg_area_co']=='FXX')]

,year,reg_code,reg_current,reg_name,reg_name_up,reg_name_lo,reg_area_co,reg_type,reg_is_ctu,reg_siren_c,geometry
0,2022,75,75,Nouvelle-Aquitaine,NOUVELLE AQUITAINE,nouvelle-aquitaine,FXX,région,Non,200053759,"MULTIPOLYGON (((-1.41248 46.18439, -1.46313 46..."
4,2022,11,11,Île-de-France,ILE DE FRANCE,île-de-france,FXX,région,Non,237500079,"POLYGON ((3.48502 48.85185, 3.48518 48.82524, ..."
5,2022,32,32,Hauts-de-France,HAUTS DE FRANCE,hauts-de-france,FXX,région,Non,200053742,"POLYGON ((3.48502 48.85185, 3.45213 48.85614, ..."
6,2022,24,24,Centre-Val de Loire,CENTRE VAL DE LOIRE,centre-val de loire,FXX,région,Non,234500023,"POLYGON ((1.50140 48.94103, 1.50734 48.92757, ..."
10,2022,52,52,Pays de la Loire,PAYS DE LA LOIRE,pays de la loire,FXX,région,Non,234400034,"MULTIPOLYGON (((-2.33743 46.71910, -2.31817 46..."
12,2022,28,28,Normandie,NORMANDIE,normandie,FXX,région,Non,200053403,"POLYGON ((1.70435 49.23243, 1.67469 49.21166, ..."
16,2022,27,27,Bourgogne-Franche-Comté,BOURGOGNE FRANCHE COMTE,bourgogne-franche-comté,FXX,région,Non,200053726,"POLYGON ((2.93662 48.16346, 2.94025 48.18297, ..."
17,2022,53,53,Bretagne,BRETAGNE,bretagne,FXX,région,Non,233500016,"MULTIPOLYGON (((-3.42742 47.64103, -3.41803 47..."
18,2022,76,76,Occitanie,OCCITANIE,occitanie,FXX,région,Non,200053791,"MULTIPOLYGON (((-0.07520 43.30716, -0.11063 43..."
21,2022,44,44,Grand Est,GRAND EST,grand est,FXX,région,Non,200052264,"POLYGON ((3.41474 48.39019, 3.42247 48.41270, ..."


The coordinates are inverted for Audun le roman.

In [2]:
lat=stops.loc[stops['stop_name']=='Audun-le-Roman-JJ.','stop_lat'][2388]
lon=stops.loc[stops['stop_name']=='Audun-le-Roman-JJ.','stop_lon'][2388]
stops.loc[stops['stop_name']=='Audun-le-Roman-JJ.','stop_lon']=lat
stops.loc[stops['stop_name']=='Audun-le-Roman-JJ.','stop_lat']=lon

Here is a function to find the region of each station in the dataframe. A similar function will be used to find in which departement and communes are the stations.

In [3]:
def where(row,areas):
    p=Point(row['stop_lon'],row['stop_lat'])
    m=areas.apply(lambda x: p.within(x['geometry']),axis=1)
    r=areas[m].reg_name.sum()
    return r
    

Let's apply this function to create a new region column for each stop.

In [4]:

stops['region']=stops.apply(lambda x : where(x,region),axis=1)

In [5]:
stops['region']

0                 Île-de-France
1                     Normandie
2                     Normandie
3                     Normandie
4                     Normandie
                 ...           
3831    Bourgogne-Franche-Comté
3832    Bourgogne-Franche-Comté
3833    Bourgogne-Franche-Comté
3834    Bourgogne-Franche-Comté
3835                   Bretagne
Name: region, Length: 3834, dtype: object

In [18]:

dpt = gpd.read_file('georef-france-departement-millesime/georef-france-departement-millesime.shp',encoding='utf-8')
dpt=dpt.loc[dpt['year']=='2022']

In [19]:
def wheredpt(row):
    p=Point(row['stop_lon'],row['stop_lat'])
    dep=dpt.loc[dpt['reg_name']==row['region']]
    m=dep.apply(lambda x: p.within(x['geometry']),axis=1)
    r=dep[m].dep_name.sum()
    return r

stops['dpt']=stops.apply(wheredpt,axis=1)

In [20]:
stops.loc[stops['region']=='Occitanie']

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,region,dpt
167,87784686,Gare de Villefranche-Vernet-l.B,NaN,42.591468,2.370015,NaN,NaN,1,NaN,Occitanie,Pyrénées-Orientales
168,87784702,Gare de Serdinya,NaN,42.567048,2.323390,NaN,NaN,1,NaN,Occitanie,Pyrénées-Orientales
169,87784710,Gare de Joncet,NaN,42.561757,2.311354,NaN,NaN,1,NaN,Occitanie,Pyrénées-Orientales
170,87784728,Gare de Olette-Canaveilles-l.B.,NaN,42.554664,2.272553,NaN,NaN,1,NaN,Occitanie,Pyrénées-Orientales
171,87784736,Gare de Nyers,NaN,42.542267,2.262999,NaN,NaN,1,NaN,Occitanie,Pyrénées-Orientales
...,...,...,...,...,...,...,...,...,...,...,...
3721,87781542,Gare de Ceilhes-Roqueredonde,NaN,43.812131,3.155299,NaN,NaN,1,NaN,Occitanie,Hérault
3722,87781559,Gare de Les Cabrils,NaN,43.778965,3.186163,NaN,NaN,1,NaN,Occitanie,Hérault
3723,87781575,Gare de Lunas,NaN,43.710023,3.194443,NaN,NaN,1,NaN,Occitanie,Hérault
3724,87781583,Gare de Le Bousquet-d'Orb,NaN,43.691547,3.168385,NaN,NaN,1,NaN,Occitanie,Hérault


In [21]:
communes=gpd.read_file('georef-france-commune.geojson',encoding='utf-8')


In [22]:
stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,region,dpt
0,87381509,Gare de Mantes-la-Jolie,NaN,48.989687,1.703294,NaN,NaN,1,NaN,Île-de-France,Yvelines
1,87415604,Gare de Vernon-Giverny,NaN,49.091286,1.478363,NaN,NaN,1,NaN,Normandie,Eure
2,87415620,Gare de Gaillon-Aubevoye,NaN,49.174632,1.352518,NaN,NaN,1,NaN,Normandie,Eure
3,87415877,Gare de Val-de-Reuil,NaN,49.275399,1.224609,NaN,NaN,1,NaN,Normandie,Eure
4,87411207,Gare de Oissel,NaN,49.343042,1.101821,NaN,NaN,1,NaN,Normandie,Seine-Maritime
...,...,...,...,...,...,...,...,...,...,...,...
3831,87718403,Gare de Roche-lez-Beaupré,NaN,47.276417,6.111272,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs
3832,87718411,Gare de Novillars,NaN,47.283906,6.132791,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs
3833,87718429,Gare de Deluz,NaN,47.292454,6.199545,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs
3834,87718437,Gare de Laissey,NaN,47.298881,6.233690,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs


In [23]:
def wherecom(row):
    p=Point(row['stop_lon'],row['stop_lat'])
    com=communes.loc[communes['dep_name']==row['dpt']]
    m=com.apply(lambda x: p.within(x['geometry']),axis=1)
    r=com[m].com_name.sum()
    return r

stops['communes']=stops.apply(wherecom,axis=1)

In [24]:
stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,region,dpt,communes
0,87381509,Gare de Mantes-la-Jolie,NaN,48.989687,1.703294,NaN,NaN,1,NaN,Île-de-France,Yvelines,Mantes-la-Jolie
1,87415604,Gare de Vernon-Giverny,NaN,49.091286,1.478363,NaN,NaN,1,NaN,Normandie,Eure,Vernon
2,87415620,Gare de Gaillon-Aubevoye,NaN,49.174632,1.352518,NaN,NaN,1,NaN,Normandie,Eure,Le Val d'Hazey
3,87415877,Gare de Val-de-Reuil,NaN,49.275399,1.224609,NaN,NaN,1,NaN,Normandie,Eure,Val-de-Reuil
4,87411207,Gare de Oissel,NaN,49.343042,1.101821,NaN,NaN,1,NaN,Normandie,Seine-Maritime,Oissel
...,...,...,...,...,...,...,...,...,...,...,...,...
3831,87718403,Gare de Roche-lez-Beaupré,NaN,47.276417,6.111272,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs,Roche-lez-Beaupré
3832,87718411,Gare de Novillars,NaN,47.283906,6.132791,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs,Novillars
3833,87718429,Gare de Deluz,NaN,47.292454,6.199545,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs,Deluz
3834,87718437,Gare de Laissey,NaN,47.298881,6.233690,NaN,NaN,1,NaN,Bourgogne-Franche-Comté,Doubs,Laissey


Some stops like the ones that are near the sea are not in the communes area defined by the opendatasoft dataset, so we will set the 'communes' value as the 'stop_name' value after 'Gare de ' 

In [25]:
missing=stops[stops['communes']==0]

In [26]:
com=communes['com_name'].tolist()

Some of the stations don't seems to be in france as the communes after "Gare de " is not in our dataset. Here we can see Monaco-Monte-Carlo is not in France but in Monaco.

In [27]:
m=missing["stop_name"].map(lambda x: x.replace("Gare de ","") not in com)
missing[m]

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,region,dpt,communes
238,87756403,Gare de Monaco-Monte-Carlo,NaN,43.738483,7.419670,NaN,NaN,1,NaN,Provence-Alpes-Côte d'Azur,0,0
239,87756478,Gare de Carnolès,NaN,43.761986,7.481441,NaN,NaN,1,NaN,0,0,0
260,85010082,Gare de Genève,NaN,46.210558,6.143155,NaN,NaN,1,NaN,0,0,0
333,87598219,Gare de Le Locle-Col-des-Roches,NaN,47.049890,6.725831,NaN,NaN,1,NaN,0,0,0
334,85043166,Gare de Le Locle,NaN,47.057880,6.746129,NaN,NaN,1,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2555,85010009,Gare de Zimeysa,NaN,46.221260,6.065743,NaN,NaN,1,NaN,0,0,0
2556,85010066,Gare de Vernier-Meyrin,NaN,46.220734,6.093892,NaN,NaN,1,NaN,0,0,0
2557,85010074,Gare de Vernier,NaN,46.220707,6.093890,NaN,NaN,1,NaN,0,0,0
2750,87734343,Gare de Chapeauroux,NaN,44.838075,3.741826,NaN,NaN,1,NaN,Auvergne-Rhône-Alpes,Haute-Loire,0


There does not seem to be any stations in those 111 stations that are not in the city after "Gare de " so we can make the communes column equal to the city.

In [28]:
stops.loc[stops['communes']==0,'communes']=stops[stops['communes']==0]["stop_name"].map(lambda x: x.replace("Gare de ",""))

# Add area in Neo4J


In [29]:
region.to_csv('region.csv')

In [30]:
dpt.to_csv('departement.csv')

In [31]:
communes['station']=communes.apply(lambda x : stops.loc[stops['communes']==x['com_name'],'stop_name'].tolist(), axis=1)

In [32]:
communes.to_csv('communes.csv')

In [33]:
stops.to_csv('station.csv')

### Were are going to had the region and the departements in Neo4j as it will be needed to differentiate communes with the same name and eventually to implement pathfinding with geolocalisation.
Add csv in dbms import directory

Region :

```
LOAD CSV WITH HEADERS FROM "file:///region.csv" AS row
MERGE (r:Region:Area {name: row.reg_name, code: row.reg_code, geometry: row.geometry})
```

Departement :

```
LOAD CSV WITH HEADERS FROM "file:///departement.csv" AS row
Match (r:Region {name: row.reg_name})
MERGE (d:Departement:Area {name: row.dep_name, code: row.dep_code, geometry: row.geometry})
Merge (d)-[:IS_IN]-(r)
```

Communes: 

```
LOAD CSV WITH HEADERS FROM "file:///communes.csv" AS row
Match (d:Departement{name: row.dep_name})
MERGE (c:Communes:Area {name: row.com_name, geometry: row.geometry})
Merge (c)-[:IS_IN]-(d)
```

```
LOAD CSV WITH HEADERS FROM "file:///station.csv" AS row
merge (s:Station {name: row.stop_name, lat:toFloat(row.stop_lat),lon:toFloat(row.stop_lon), id:row.stop_id, location_type:row.location_type}) With s, row
Match (c:Communes {name: row.communes})-[IS_IN]-(d:Departement {name:row.dpt})
Merge (c)-[:HAS_TRAIN_STATION]-(s)
```